In [42]:

import openai
import json
from pykrx import stock
from pykrx import bond

TICKER_SAMSUNG = "005930"

SYSTEM_PROMPT = """
            너는 삼성전자 주식 정보를 알려주는 봇이야. 
            아래와 같은 질문이 온 경우에만 대답해야해
            1. 삼성 또는 삼성전자의 주식 정보
            2. 날짜를 지정해야 함, 특정 연도로 말한경우 해당 연도 1월부터 12월까지 전체, 월을 지정한 경우 해당 월 1일부터 말일까지
            
            CASE1
            질문이 올바르지 않은 경우 아래와 같은 대답을 출력해줘
            1. 삼성 또는 삼성전자가 아닌 다른 회사의 주식 정보를 물어본 경우
              -> 저는 삼성전자의 주식 정보만 대답할 수 있습니다.
            2. 날짜의 범위를 설정하지 않은 경우
              -> 날짜의 범위를 지정해서 다시 질문해 주세요.
            3. 그 외 엉뚱한 질문인 경우
              -> 무슨 말인지 모르겠어요, 삼성전자의 주식 정보를 날짜 범위를 지정해 질문해 주세요.

            CASE2
            질문이 올바른 경우 아래와 같은 JSON 형식으로 응답해줘
            {
                start_dt: 사용자가 말한 시작일
                end_dt: 사용자가 말한 종료일
            } 
"""

def is_valid_json(data):
    try:
        json.loads(data)
        return True
    except (TypeError, ValueError):
        return False

def get_market_fundamental(start_dt, end_dt, ticket):
    df = stock.get_market_fundamental(start_dt, end_dt, ticket)
    return df.head(365)

def get_market_ohlcv(start_dt, end_dt, ticket):
    df = stock.get_market_ohlcv(start_dt, end_dt, ticket)
    return df.head(365)

def chat_create(question, prompt):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user", 
                "content": question
            }],
        )
    return response.choices[0].message.content

def ask(question, prompt, cache):
    if(prompt != ""):
       cache = chat_create(question, prompt)
       return ask(question, "", cache)
    else:
        if not is_valid_json(cache):
            return cache

        ref = json.loads(cache)
        start_dt = ref["start_dt"]
        end_dt = ref["end_dt"]
        fundamental = get_market_fundamental(start_dt, end_dt, TICKER_SAMSUNG)
        ohlcv = get_market_ohlcv(start_dt, end_dt, TICKER_SAMSUNG)

        fundamental_dict = fundamental.to_dict(orient="records")
        ohlcv_dict = ohlcv.to_dict(orient="records")

        fundamental_str = json.dumps(fundamental_dict, ensure_ascii=False)
        ohlcv_str = json.dumps(ohlcv_dict, ensure_ascii=False)

        question = (
            "질문과 데이터를 바탕으로 분석해서 결과를 알려줘"
            + "질문 : " + question
            + ", 데이터1 : " + fundamental_str
            + ", 데이터2 : " + ohlcv_str
        )
        return chat_create(question, "")

def chat():
    print("삼성전자의 주식 정보에 대해 날짜 범위와 함께 질문해 주세요.")
    while True:
        user_input = input("User: ")
        print(f"USER: {user_input}")

        if user_input.lower() == 'exit':
            print("Exiting...")
            break
        response = ask(user_input, SYSTEM_PROMPT, "")
        print(f"GPT: {response}")

if __name__ == "__main__":
    chat()


삼성전자의 주식 정보에 대해 날짜 범위와 함께 질문해 주세요.
USER: 삼전 2024년 주식 정보 요약
GPT: 날짜의 범위를 지정해서 다시 질문해 주세요.
USER: 
GPT: 무슨 말인지 모르겠어요, 삼성전자의 주식 정보를 날짜 범위를 지정해 질문해 주세요.
USER: 삼성 2024년 주식 정보 요약
GPT: 날짜의 범위를 지정해서 다시 질문해 주세요.
USER: 
GPT: 무슨 말인지 모르겠어요, 삼성전자의 주식 정보를 날짜 범위를 지정해 질문해 주세요.
USER: 2024년의 삼성전자 주식 정보를 3줄로 요약해줘
GPT: 무슨 말인지 모르겠어요, 삼성전자의 주식 정보를 날짜 범위를 지정해 질문해 주세요.
USER: 삼성전자 2024년 1월 주식 정보 요약해줘
GPT: 삼성전자의 2024년 1월 주식 정보를 요약하면 다음과 같습니다.

### 재무 비율
1. **BPS (주당순자산)**: 50,817원
2. **PER (주가수익비율)**: 평균 9.33
   - 최저: 8.81
   - 최고: 9.88
3. **PBR (주가순자산비율)**: 평균 1.46
   - 최저: 1.40
   - 최고: 1.57
4. **EPS (주당순이익)**: 8,057원
5. **DIV (배당수익률)**: 평균 1.89%
   - 최저: 1.81%
   - 최고: 2.03%
6. **DPS (주당배당금)**: 1,444원

### 주가 및 거래량
- **거래일 기준 시가/고가/저가/종가 및 거래량**:
   - 최근 20일의 평균 시가: 약 73700원
   - 최근 20일의 평균 종가: 약 74300원
   - 거래량: 2024년 1월 평균 거래량은 여러 일자를 고려할 때 약 17,143,000주
   - 최근 등락률: 최근 20일 중 일부 날짜에서 변동성이 있음 (등락률 -3.27%에서 +4.18%까지)

### 주요 분석
- **PER**은 9.33으로, 일반적으로 주식이 저평가된 것으로 해석 가능하다.
- **PBR**은 1.46로, 자산 대비 주가가 적정선에 위치

In [40]:
df = stock.get_market_fundamental("2024-12-25", "2024-12-31", "005930")
print(df.head(31))

              BPS    PER   PBR   EPS   DIV   DPS
날짜                                              
2024-12-26  52002  25.15  1.03  2131  2.69  1444
2024-12-27  52002  25.20  1.03  2131  2.69  1444
2024-12-30  52002  24.96  1.02  2131  2.71  1444
